In [1]:
from lib.Environment import DeepTradingEnvironment
import datetime
# parameters related to the transformation of data, this parameters govern an step before the algorithm
meta_parameters = {"in_bars_count": 30,
                   "out_reward_window": datetime.timedelta(days=7),
                   "state_type":"in_window_out_window"}

# parameters that are related to the objective/reward function construction
objective_parameters = {"percent_commission": .001}
print("===Meta Parameters===")
print(meta_parameters)
print("===Objective Parameters===")
print(objective_parameters)


assets_simulation_details={"asset_1":{"method":"GBM","sigma":.1,"mean":.1},
                    "asset_2":{"method":"GBM","sigma":.2,"mean":.2}}

env=DeepTradingEnvironment.build_environment_from_simulated_assets(assets_simulation_details=assets_simulation_details,
                                                                     data_hash="simulation_gbm",
                                                                     meta_parameters=meta_parameters,
                                                                     objective_parameters=objective_parameters)



===Meta Parameters===
{'in_bars_count': 30, 'out_reward_window': datetime.timedelta(days=7), 'state_type': 'in_window_out_window'}
===Objective Parameters===
{'percent_commission': 0.001}


In [2]:
# get state data

date=env.features.index[100]
state_features, weights_on_date =env.state.get_state_on_date(target_date=date)
print("on date {} we observe {} input features".format(date,state_features.shape))
state_features

on date 2021-01-04 17:20:00 we observe (30, 2) input features


,asset_1_log_returns,asset_2_log_returns
2020-12-06 17:20:00,-0.002495,-0.007164
2020-12-07 17:20:00,-0.008014,-0.026163
2020-12-08 17:20:00,0.003765,-0.004675
2020-12-09 17:20:00,-0.001883,-0.005627
2020-12-10 17:20:00,-0.001403,0.016586
2020-12-11 17:20:00,-0.004107,-0.014523
2020-12-12 17:20:00,0.009391,0.015092
2020-12-13 17:20:00,0.016332,-0.004943
2020-12-14 17:20:00,0.002346,-0.009137
2020-12-15 17:20:00,-0.001146,0.000893


In [3]:
print("and the weights that the portfolio have up to that time")
weights_on_date.head()

and the weights that the portfolio have up to that time


asset_1_forward_return_7_days,_0:00:00    0.5
asset_2_forward_return_7_days,_0:00:00    0.5
Name: 2021-01-04 17:20:00, dtype: float64

In [4]:
action_portfolio_weights = np.random.rand(2)
obs, reward, done, info = env.step(action_portfolio_weights=action_portfolio_weights,
                                                action_date=date)
reward

2021-01-04 17:20:00    0.006
Freq: 1440T, dtype: float64

# Linear Policy Parametrization

## Continouse Policies for Continuous Actions

### Background

Gaussian probability distribution:

$$
p(x):=\frac{1}{\sigma\sqrt{2\pi}}exp(-\frac{(x-\mu)^2}{2\sigma^2})
$$

### Gaussian Policy

$$
\pi(a |s,\boldsymbol\theta):=\frac{1}{\sigma(s,\boldsymbol\theta)\sqrt{2\pi}}exp(-\frac{(a-\mu(s,\boldsymbol\theta))^2}{2\sigma(s,\boldsymbol\theta)^2})
$$

#### Linear Parametrization

$$
\mu(s,\boldsymbol\theta) :=\boldsymbol{\theta_{\mu}}^T \pmb{x}(s)
$$

$$
\sigma(s,\boldsymbol\theta) :=exp(\boldsymbol{\theta_{\sigma}}^T \pmb{x}(s))
$$